In [1]:
%matplotlib notebook 
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import HBox, Label
from ipywidgets import IntSlider

# pretty print all cell's output and not just the last one
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

Summary from all of the different notes we have:

We have the Gaussian potential $V(x)=-U\sum_n e^{-(x-na)^2/2\sigma^2}$ which is writen in Fourier components as: $V(x)=\sum_{G_j}-U\sqrt {2\pi}\frac{\sigma}{a}e^{-G^2_j \sigma^2/2} e^{i G_j x} \equiv  \sum_{G_j} V_{G_j} \cdot e^{i G_j x}$.

We are solving the eigenvalue equation:
$\Big( \frac{\hbar^2 (k+K)^2}{2m} - \varepsilon \Big) c_{k+K} +\sum_G c_{k+K-G} V_G$ and search for $\varepsilon$ - the energy eigenvectors.

Define different stuff
* Don't confuse, *K* is a reciprocal lattice vector, *k* is a wave vector $k\in [-\pi/a; \pi/a)$
* *MaxK* - maximal reciprocal lattice vector we consider including in the sum ($K$ or $K-G$). Then summation goes from *-MaxK* to *MaxK*
* *N_k* - in how many peaces we split $[0, \pi/a)$
* Potential here and energy later at the moment is in arbitrary units, but need to be calculated. I believe I could make it all pretier by multiplying only at the end with some constant which would bring units. Otherwise it does not look good now

In [76]:
a=3   #Latice constant in Åmstrongs
sigma=0.5 #Very arbitrary value
U=10     #
maxK=6  
NrOfK=maxK*2+1
Len_k=15



The ampliture of $V_G$ we call $A$, thus $A \equiv -U\sqrt {2\pi}\frac{\sigma}{a}$ and  $V_G = A e^{-G^2 \sigma^2/2} $

In [69]:
A=-U*np.sqrt(2*sigma)*np.sqrt(2 *np.pi)*sigma/a

Run Bandstructure file for each *k* value

In [70]:
Energies=np.zeros(shape=(Len_k*2+1, NrOfK))
kVect=[0]*(Len_k*2+1) #Vector for plotting containing all of the k values in 1/Angstrom
for ki in range(-Len_k,Len_k+1):
    k=np.pi/(2*a)*ki/Len_k  #I am a bit confused why /2a
    %run BandStructureFunction.ipynb
    Energies[ki+Len_k]=np.real(E)
    kVect[ki+Len_k]=k #in 1/Angstrom

Plot potentials

In [73]:
%%capture
from matplotlib import pyplot as plt
%matplotlib notebook 
%matplotlib notebook 
#If you want to display the figure here, then uncomment previous line
fig, [ax,ax2]=plt.subplots(nrows=1, ncols=2)  #Two plots in the same figure

x=np.linspace(-5*a,5*a,401)  
#length of the x vector should be an odd number so that 0 is included,
#otherwise it does not plot the peak correctly
Vx=-U*np.exp(-x**2/(2*sigma**2))
VxSum=[0]*len(x)
for n in range(-5,6):
    Vx1=-U*np.exp(-(x-n*a)**2/(2*sigma**2))
    ax.plot(x,Vx1,'y--')
    VxSum=VxSum+Vx1
ax.plot(x,VxSum)
ax.plot(x,Vx)
ax.set(xlabel='a, $\AA$',ylabel='V, arb. u', title='Atomic potential')

Plot energies

In [72]:
%%capture
ax2.plot(kVect,Energies[:,0:5], color='purple')
ax2.set(xlabel='k, $1/ \AA$',ylabel='E, arb. u', title='Band structure')

In [41]:
fig.set_size_inches(9.5, 3.5)
fig.subplots_adjust(wspace=0.4, bottom=0.2) #Margins around the subplots

In [42]:
import ipywidgets as widgets
from ipywidgets import interact, interactive
from IPython.display import display

#fig, [ax,ax2]=plt.subplots(nrows=1, ncols=2)  #Two plots in the same figure

In [45]:
def f(Lattice_Constant=a, Amplitude=U, NumKVec=maxK, PlotK=5):
    global a,A, k, Len_k, maxK, U, NrOfK  #So that BandStructureFunction.ipynb knows the variables used in a function f
    a=Lattice_Constant
    U=Amplitude
    maxK=NumKVec
    #sigma=0.5 #Very arbitrary value
    A=-U*np.sqrt(2 *np.pi)*sigma/a #np.sqrt(2*sigma)
    NrOfK=maxK*2+1
    Energies=np.zeros(shape=(Len_k*2+1, NrOfK))
    kVect=[0]*(Len_k*2+1)
    for ki in range(-Len_k,Len_k+1):
        k=np.pi/(2*a)*ki/Len_k  #I am a bit confused why /2a
        %run BandStructureFunction.ipynb
        Energies[ki+Len_k]=np.real(E)
        kVect[ki+Len_k]=k
    ax.cla()
    x=np.linspace(-5*a,5*a,401)
    Vx=-U*np.exp(-x**2/(2*sigma**2))
    VxSum=[0]*len(x)
    for n in range(-5,6):
        Vx1=-U*np.exp(-(x-n*a)**2/(2*sigma**2))
        ax.plot(x,Vx1,'y--')
        VxSum=VxSum+Vx1
        
  
    ax.plot(x,VxSum)
    ax.plot(x,Vx)
    ax.set(xlabel='a, $\AA$',ylabel='V, eV', title='Atomic potential')
   
    ax2.cla()
    ax2.plot(kVect, Energies[:,0:PlotK],color='purple')
    ax2.set(xlabel='k, $1/ \AA$',ylabel='E, eV', title='Band structure')
    
    #return Energies

The_Interaction=interactive(f,Lattice_Constant=(1,10,0.5),Amplitude=(0,50,1), NumKVec=(1,20,1),PlotK=(0,10,1) ) 

In [44]:
for widg in The_Interaction.children[:-1]:
    widg.description = ""
    widg.continuous_update = False
    

Lattice_Const, Potential_Amp, Num_Of_K, PlotK = [The_Interaction.children[i] for i in range(4)]

#display(Lattice_Const,Potential_Amp ,Num_Of_K )
FirstBox = widgets.HBox([Label(r'Lattice constant'), Lattice_Const, Label(r'Potential depth'), Potential_Amp,])
SecondBox = widgets.HBox([ Label (r'N of k vectors'), Num_Of_K, Label (r'How many bands to plot'), PlotK])

We can calculate the theoretical minimum of the first band

In [81]:
V0=U*5  *1.9e-19 #eV, potential depth
E0 = hbar/sigma*np.sqrt(V0/(m)) *1e10/(1.9e-19)

In [82]:
E0

359.4228737144295